In [25]:
# Task 7.1
class File:
    def __init__(self, file_name, method):
        self.file_obj = open(file_name, method)

    def __enter__(self):
        print("Succesful open!")
        return self.file_obj

    def __exit__(self, exc_type, value, traceback):
        if type is not None:
            print("Exception has been handled: ")
            print("An exception occurred in your with block:", exc_type)
            print("Exception message:", value)
            print("Traceback:", traceback)
        self.file_obj.close()
        print("Succesful close!")
        return True


with File("my_true_file_1.txt", "w") as f:
    f.write("Succes!")
print()
with File("my_false_file_1.txt", "w") as f:
    f.writ("Error!") # Exception: AttributeError

Succesful open!
Succesful close!

Succesful open!
Exception has been handled: 
An exception occurred in your with block: <class 'AttributeError'>
Exception message: '_io.TextIOWrapper' object has no attribute 'writ'
Traceback: <traceback object at 0x000001A9B23C70C0>
Succesful close!


In [31]:
# Task 7.2
from contextlib import contextmanager

@contextmanager
def work_with_file(file_name, method):
    file = None
    try:
        print("Opening file")
        file = open(file_name, method)
        yield file
    except OSError:
        print("We had an error!")
    finally:
        print("Closing file")
        if file:
            file.close()
        

with work_with_file("my_true_file_2.txt", "r") as f:
    print(f.read())
print()
with work_with_file("my_false_file_2.txt", "R") as f: # Error
    print(f.read())

Opening file
Some text...
Closing file

Opening file
Closing file


ValueError: invalid mode: 'R'

In [5]:
# Task 7.3
from time import perf_counter
from contextlib import ContextDecorator


class time_logger(ContextDecorator):
    def __init__(self, file_name):
        self.file_name = file_name

    def __enter__(self):
        self.start = perf_counter()

    def __exit__(self, exc_type, value, traceback):
        log_file = open(self.file_name, "a")
        log_file.write(f"Compiled in {perf_counter() - self.start} sec\n")
        log_file.close()


@time_logger("log_file.txt")
def pow_func(a, b):
    return a ** b


def mult_func(a, b):
    return a * b


pow_func(5000000, 100000)
with time_logger("log_file.txt"):
    mult_func(5000000, 100000)
    
with open("log_file.txt", "r") as f:
    print(f.read())

Compiled in 0.1137796000000435 sec
Compiled in 0.00034990000000334476 sec



In [19]:
# Task 7.4
from contextlib import suppress


def ignore_exc(func):
    def wrapper(*args, **kwargs):
        with suppress(Exception):
            result = func(*args, **kwargs)
            print("Exception not occure!")
            return result
    return wrapper


@ignore_exc
def func_1(a, b):
    return a / b


@ignore_exc
def func_2(my_list, index):
    return my_list[index]


@ignore_exc
def func_3(my_dict, key):
    return my_dict[key]


func_1(10, 0) 

In [20]:
func(10, 1)

Exception not occure!


10.0

In [21]:
func_2(['a', 'b', 'c'], 4)

In [22]:
func_2(['a', 'b', 'c'], 1)

Exception not occure!


'b'

In [23]:
func_3({"cat": "Tom"}, "not_key")

In [24]:
func_3({"cat": "Tom"}, "cat")

Exception not occure!


'Tom'

In [2]:
# Task 7.5
class EvenError(ValueError):
    pass


class NotEvenValue(EvenError):
    pass


class NotIntValue(EvenError):
    pass


class NotGreaterThanTwo(EvenError):
    pass


def func(number):
    if not isinstance(number, int):
        raise NotIntValue("Number isn`t integer type!")
    if number < 3:
        raise NotGreaterThanTwo("Number isn`t greater than 2!")
    if number % 2 != 0:
        raise NotEvenValue("Number isn`t even!")
    return True
        

func(1.01)

NotIntValue: Number isn`t integer type!

In [9]:
func(2)

NotGreaterThanTwo: Number isn`t greater than 2!

In [10]:
func(3)

NotEvenValue: Number isn`t even!

In [13]:
func(4)

True

In [8]:
# Task 7.6
def goldbach_problem(number):
    func(number) # function from Task 7.5
    a = 2
    b = None
    while True:
        if is_prime_number(a):
            b = number - a
            if is_prime_number(b):
                return print(f"{number} = {a} + {b}")
        a += 1

        
def is_prime_number(number):
    for i in range(2, number):
        if number % i == 0:
            return False
    return True


while True:
    if (number := input("Enter your number: ")) != 'q':
        goldbach_problem(int(number))
    else:
        break

Enter your number: 10
10 = 3 + 7
Enter your number: 64
64 = 3 + 61
Enter your number: 124
124 = 11 + 113
Enter your number: q


In [42]:
# Task 7.7
class MyNumberCollection:
    @staticmethod
    def check_element_type(element):
        if not isinstance(element, int):
            raise TypeError(f"{MyNumberCollection.__name__} supports only numbers! \
            {type(element)}({element}) - object isn`t a number!")

        
        
    @staticmethod
    def create_collection(start, stop, step):
        collection_list = []
        if isinstance(start, tuple) or isinstance(start, list):
            for el in start:
                MyNumberCollection.check_element_type(el)
            collection_list = list(start)
        else:
            collection_list = [x for x in range(start, stop, step)]
            if stop not in collection_list:
                collection_list.append(stop)
        return collection_list
        
    
    def append(self, number):
        MyNumberCollection.check_element_type(number)
        self.collection.append(number)
    
    
    def __init__(self, start, stop=None, step=None):
        self.collection = self.create_collection(start, stop, step)
        
        
    def __repr__(self):
        return str(self.collection)
    
    def __add__(self, other):
        new_collection_list = self.collection + other.collection
        return MyNumberCollection(new_collection_list)
    
    def __getitem__(self, index):
        if not isinstance(index, int):
            raise TypeError("The argument index has to be integer")
        if index >= len(self.collection):
            raise IndexError("Index out of range")
        return self.collection[index]**2
    
    def __iter__(self):
        self.__n = 0
        return self
    
    def __next__(self):
        if self.__n < len(self.collection):
            result = self.collection[self.__n]
            self.__n += 1
            return result
        else:
            raise StopIteration
        
    

In [43]:
col1 = MyNumberCollection(0, 5, 2)
print(col1)

[0, 2, 4, 5]


In [44]:
col2 = MyNumberCollection((1,2,3,4,5))
print(col2)

[1, 2, 3, 4, 5]


In [45]:
col3 = MyNumberCollection((1,2,3,"4",5))

TypeError: MyNumberCollection supports only numbers!             <class 'str'>(4) - object isn`t a number!

In [46]:
col1.append(7)
print(col1)

[0, 2, 4, 5, 7]


In [47]:
col2.append("string")

TypeError: MyNumberCollection supports only numbers!             <class 'str'>(string) - object isn`t a number!

In [48]:
print(col1 + col2)

[0, 2, 4, 5, 7, 1, 2, 3, 4, 5]


In [49]:
print(col1)

[0, 2, 4, 5, 7]


In [50]:
print(col2)

[1, 2, 3, 4, 5]


In [51]:
print(col2[4])

25


In [52]:
print(col2[10])

IndexError: Index out of range

In [53]:
for item in col1:
    print(item, end=" ")

0 2 4 5 7 

In [11]:
# Task 7.8
class MySquareIterator:
    def __init__(self, numbers):
        self.numbers = numbers
        self.stop = len(numbers)
        
    def __iter__(self):
        self.n = 0
        return self
    
    def __next__(self):
        if self.n < self.stop:
            result = self.numbers[self.n]**2
            self.n += 1
            return result
        else:
            raise StopIteration
            
            
lst = [1, 2, 3, 4, 5]
itr = MySquareIterator(lst)
for item in itr:
    print(item)

1
4
9
16
25


In [56]:
# Task 7.9
class EvenRange:
    def __init__(self, start, stop):
        self.start = start if start % 2 == 0 else start + 1
        self.stop = stop
        self.n = self.start
    
    def __iter__(self):
        return self
    
    def __next__(self):
        result = self.n
        if self.n >= self.stop:
            print("Out of numbers!")
            raise StopIteration("Out of numbers!")
        self.n += 2
        return result
        
        
er1 = EvenRange(7,11)
next(er1)

8

In [57]:
next(er1)

10

In [58]:
next(er1)

Out of numbers!


StopIteration: Out of numbers!

In [59]:
next(er1)

Out of numbers!


StopIteration: Out of numbers!

In [55]:
er2 = EvenRange(3, 14)
for number in er2:
    print(number, end = " ")

4 6 8 10 12 Out of numbers!


In [60]:
# Task 7.10
from time import sleep


def endless_generator(odd=1):
    while True:
        yield odd
        odd += 2


gen = endless_generator()
while True:
    print(next(gen))
    sleep(1)

1
3
5
7
9
11
13
15
17
19
21
23
25
27
29
31
33
35
37
39
41
43
45
47
49
51
53
55
57
59


KeyboardInterrupt: 

In [62]:
# Task 7.11
def endless_fib_generator():
    num_1, num_2 = 0, 1
    yield num_2
    while True:
        num_1, num_2 = num_2, num_1 + num_2
        yield num_2
        
        
gen = endless_fib_generator()
while True:
    print(next(gen))
    sleep(1)

1
1
2
3
5
8
13
21
34
55
89
144
233
377
610
987
1597
2584
4181
6765
10946
17711
28657
46368
75025
121393
196418


KeyboardInterrupt: 